In [53]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [54]:
nvi_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/Temp_NVIZones2026v2__20250226/Temp_NVIZones2026v2__20250226.shp")
detroit_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/City_of_Detroit_Boundary/City_of_Detroit_Boundary.shp")


In [55]:
# Load birth data from csv
births_df = pd.read_csv('Q:/Vital Records/2023/2023 Birth Files.csv')


C:\Users\abhi\AppData\Local\Temp\ipykernel_22836\1927482806.py:2: DtypeWarning: Columns (0,36) have mixed types. Specify dtype option on import or set low_memory=False.
  births_df = pd.read_csv('Q:/Vital Records/2023/2023 Birth Files.csv')


In [56]:
# Assuming the birth data contains 'latitude' and 'longitude' columns
# Convert to a GeoDataFrame
births_gdf = gpd.GeoDataFrame(
    births_df, 
    geometry=gpd.points_from_xy(births_df.LONGITUDE, births_df.LATITUDE), 
    crs="EPSG:4326")

### Detroit - City Wide

In [57]:
# Ensure both GeoDataFrames have the same CRS
births_gdf = births_gdf.to_crs(detroit_shp.crs)

# Perform a spatial join to assign births to geographic areas
births_with_geography = gpd.sjoin(births_gdf, detroit_shp, how="left", predicate="within")

#No of rows in the data
total_births = births_with_geography.groupby("name")["KESSNER"].count().reset_index()
total_births.columns = ["name", "total_births"]

# Filter for births where kesser == 1
births_kesser_1 = births_with_geography[births_with_geography["KESSNER"] == 1]

# Aggregate count of kesser = 1 by geographic area (e.g., 'zone' column from shapefile)
adequate_care_counts = births_kesser_1.groupby("name")["KESSNER"].count().reset_index()
adequate_care_counts.columns = ["name", "kessner_1_count"]

births_summary = total_births.merge(adequate_care_counts, on="name", how="left")
births_summary["percentage_adequate"] = (births_summary["kessner_1_count"] / births_summary["total_births"]) * 100

display(births_summary)

,name,total_births,kessner_1_count,percentage_adequate
0,Detroit,7426,4594,61.863722


### NVI Zones

In [59]:
# Ensure both GeoDataFrames have the same CRS
births_gdf = births_gdf.to_crs(nvi_shp.crs)

# Spatial join: Assign each point to a polygon
merged_gdf = gpd.sjoin(nvi_shp, births_gdf, how="left", predicate="intersects")
print(merged_gdf)

#No of rows in the data
total_births_nvi = merged_gdf.groupby("DistrictNu")["KESSNER"].count().reset_index()
total_births_nvi.columns = ["DistrictNu", "total_births"]

# Filter for births where kesser == 1
births_kesser_1_nvi = merged_gdf[merged_gdf["KESSNER"] == 1]

# Aggregate count of kesser = 1 by geographic area (e.g., 'zone' column from shapefile)
adequate_care_counts = births_kesser_1_nvi.groupby("DistrictNu")["KESSNER"].count().reset_index()
adequate_care_counts.columns = ["DistrictNu", "kessner_1_count"]

births_summary_nvi = total_births_nvi.merge(adequate_care_counts, on="DistrictNu", how="left")
births_summary_nvi["percentage_adequate"] = (births_summary_nvi["kessner_1_count"] / births_summary_nvi["total_births"]) * 100

display(births_summary_nvi)

    FID_1  DistrictNu ZoneName    Shape__Are     Shape__Len    Shape__A_1  \
0       1           1       1c  1.801933e+08   57935.111599  1.801933e+08   
1       2           2       2a  1.657429e+08   54206.458633  1.657429e+08   
2       3           4       4a  1.841099e+08   63897.869454  1.841099e+08   
3       4           6       6a  1.852068e+08   73149.796786  1.852068e+08   
4       5           3       3a  2.069255e+08   73930.680151  2.069255e+08   
5       6           7       7c  2.133807e+08   81969.825000  2.133807e+08   
6       8           1       1a  1.892972e+08   59301.844849  1.892972e+08   
7       9           7       7a  2.026535e+08   64998.755711  2.026535e+08   
8      10           7       7b  1.236859e+08   50867.332052  1.236859e+08   
9      11           2       2c  1.484800e+08   69074.497143  1.484800e+08   
10     12           2       2b  1.616082e+08   51595.955522  1.616082e+08   
11     13           3       3c  1.933867e+08   80498.754084  1.933867e+08   

,DistrictNu,total_births,kessner_1_count,percentage_adequate
0,1,0,NaN,NaN
1,2,0,NaN,NaN
2,3,0,NaN,NaN
3,4,0,NaN,NaN
4,5,0,NaN,NaN
5,6,0,NaN,NaN
6,7,0,NaN,NaN


In [61]:
point_in_poly_count = sum(1 for point in births_gdf.geometry for poly in nvi_shp.geometry if point.intersects(poly))
print(f"Points intersecting with polygons: {point_in_poly_count} out of {len(births_gdf)}")

Points intersecting with polygons: 0 out of 99179
